# Interactive Visualization of Global Economic Freedom

Compiled by the Heritage Foundation, the Index of Economic Freedom consists of 10 quantitative and qualitative factors of economic freedom, each graded on a scale of 0 to 100. This project aims to visualize this information on an interactive map that provides a useful overview of global economic freedom and encourages users to explore the data. I employ the module plotly for this purpose, focusing primariy on the choropleth function. 

Fist import the data and filter to keep only 2016 data. My purpose is to show global overview, not time series, so I am only concerned with most recent data. 

In [1]:
import pandas as pd
data = pd.read_csv('data.csv')
data2016 = data[data['index year'] == 2016]
data2016.head()

,name,index year,overall score,property rights,freedom from corruption,fiscal freedom,government spending,business freedom,labor freedom,monetary freedom,trade freedom,investment freedom,financial freedom
0,Afghanistan,2016,NaN,NaN,12.0,91.6,81.2,56.6,63.2,NaN,NaN,55.0,NaN
1,Albania,2016,65.9,35.0,33.0,87.8,75.0,67.6,51.5,81.7,87.6,70.0,70.0
2,Algeria,2016,50.1,25.0,36.0,81.0,59.4,62.1,48.2,68.1,60.8,30.0,30.0
3,Angola,2016,48.9,15.0,19.0,87.8,50.1,50.3,44.8,72.2,70.2,40.0,40.0
4,Argentina,2016,43.8,15.0,34.0,66.1,51.3,56.0,43.9,44.0,67.4,30.0,30.0


Upon inspection, it appears that some entries for country names (such as Bangladesh) contain trailing spaces, making it impossible for plotly to recognize the countries. 

In [2]:
data2016['name'][10]

'Bangladesh '

My solution to this problem is to create a simple lambda function that strips all trailing spaces before and after a string and apply it to the column _name_.

In [3]:
data2016['name'] = data2016['name'].map(lambda x: x.strip())

/Users/SupremeLeader/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


The data is still not ready for plot. Even though there is an option for plotly to use country names to match data to map, its default list of country name does not match the name in my data. This is quite common when it comes to country data. For example, some organization would display Russia as _Russia_, and others would display as _Russian Federation_. There is no standard to this so plotly's option to plot by country names is not very reliable. The alternative of using ISO-3 codes to plot is much better since ISO-3 is a standard. 

I use the module _countrycode_ to match the country names in the data to their respective ISO-3 codes and put them all into a new column.

In [4]:
from countrycode import countrycode
data2016['code'] = data2016['name'].map(lambda x: countrycode(codes = x, origin = 'country_name', target = 'iso3c'))

/Users/SupremeLeader/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Test to see if the function works. Since our list is rather long, printing _data2016['name']_ would only show the first and last few rows. Instead, I print all column values as strings to bypass this limit.

In [5]:
print (data2016[['name','code']].to_string())

                                 name                   code
0                         Afghanistan                    AFG
1                             Albania                    ALB
2                             Algeria                    DZA
3                              Angola                    AGO
4                           Argentina                    ARG
5                             Armenia                    ARM
6                           Australia                    AUS
7                             Austria                    AUT
8                          Azerbaijan                    AZE
9                             Bahrain                    BHR
10                         Bangladesh                    BGD
11                           Barbados                    BRB
12                            Belarus                    BLR
13                            Belgium                    BEL
14                             Belize                    BLZ
15                      

It seems like the ISO-3 function does not work for three observation: Kosovo, São Tomé and Príncipe, and South Korea. The first two instances make sense since Kosovo does not have an official ISO-3 code yet, and São Tomé and Príncipe has accented characters which are not recognized by _countrycodes_. I do not know why South Korea does not work, though I suspect that _countrycodes_ only recognizes _Republic of South Korea_ which is a more official name. I manually input the codes for São Tomé and Príncipe and South Korea, and drop Kosovo from the data. Without an ISO-3 code, plotly would not recognize Kosovo's data anyhow. 

In [6]:
data2016['code'][data2016['code'] == "São Tomé and Príncipe"] = 'STP'
data2016['code'][data2016['code'] == "South Korea"] = 'KOR'
data2016 = data2016[data2016['code'] != 'Kosovo']

/Users/SupremeLeader/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/SupremeLeader/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/SupremeLeader/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2885: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_gl

The data is now ready for plotly. Set up API username and key.

In [7]:
import plotly
import plotly.plotly as py

In [8]:
plotly.tools.set_credentials_file(username='brickkiln', api_key='jd10b5299b')

Using the sample code from plotly documentation, I test the function using only _overall score_ to see how compatible the data is to plotly.

In [9]:
data1 = [ dict(
        type = 'choropleth',
        locations = data2016['code'],
        z = data2016['overall score'], # z is default for plotly.
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 1.0
            ) ),
        colorbar = dict(
            ticksuffix = ' points',
            title = 'Economic Freedom Index'),
      ) ]

layout = dict(
    title = '2016 Index of Economic Freedom<br>Source:\
            <a href="http://www.heritage.org/index/">\
            The Heritage Foundation</a>',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'orthographic'
        )
    )
)

fig = dict( data=data1, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~brickkiln/0 or inside your plot.ly account where it is named 'd3-world-map'


This looks pretty good, but we can do better. The data cover 10 freedom indices, and this only shows the overall. Ideally, we should have a dropdown menu to let users visualize the freedom indice they want. These follwing codes attempt to do precisely that. Plotly works by data traces, thus we need to have one trace for each freedom we need to plot, and put them all in a list. We also need a button for each trace that enables the visibility of that particular trace and hides the rest. This is implemented as follow. 

In [10]:
import plotly.plotly as py
from plotly.graph_objs import *


trace1 = Choropleth(
        locations = data2016['code'],
        z = data2016['overall score'],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

trace2 = Choropleth(
        locations = data2016['code'],
        z = data2016['property rights'],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

trace3 = Choropleth(
        locations = data2016['code'],
        z = data2016['freedom from corruption'],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

trace4 = Choropleth(
        locations = data2016['code'],
        z = data2016['fiscal freedom'],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

trace5 = Choropleth(
        locations = data2016['code'],
        z = data2016['government spending'],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

trace6 = Choropleth(
        locations = data2016['code'],
        z = data2016['business freedom'],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

trace7 = Choropleth(
        locations = data2016['code'],
        z = data2016['labor freedom'],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

trace8 = Choropleth(
        locations = data2016['code'],
        z = data2016['monetary freedom'],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

trace9 = Choropleth(
        locations = data2016['code'],
        z = data2016['trade freedom'],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

trace10 = Choropleth(
        locations = data2016['code'],
        z = data2016['investment freedom'],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

trace11 = Choropleth(
        locations = data2016['code'],
        z = data2016['financial freedom'],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

data = Data([trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9, trace10, trace11])

layout = Layout(
    updatemenus=list([
        dict(
            x=-0.05,
            y=1,
            buttons=
                list([
                dict(
                    args=['visible', [True, False, False, False, False, False, False, False, False, False, False]],
                    label='overall score',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, True, False, False, False, False, False, False, False, False, False]],
                    label='property rights',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, True, False, False, False, False, False, False, False, False]],
                    label='freedom from corruption',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, True, False, False, False, False, False, False, False]],
                    label='fiscal freedom',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, False, True, False, False, False, False, False, False]],
                    label='government spending',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, False, False, True, False, False, False, False, False]],
                    label='business freedom',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, False, False, False, True, False, False, False, False]],
                    label='labor freedom',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, False, False, False, False, True, False, False, False]],
                    label='monetary freedom',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, False, False, False, False, False, True, False, False]],
                    label='trade freedom',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, False, False, False, False, False, False, True, False]],
                    label='investment freedom',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, False, False, False, False, False, False, False, True]],
                    label='financial freedom',
                    method='restyle'
                )
            ]),
            yanchor='top'
        )
    ]),
    title = '2016 Index of Economic Freedom<br>Source:\
            <a href="http://www.heritage.org/index/">\
            The Heritage Foundation</a>',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'orthographic'
        )
        ),
)
fig = Figure(data=data, layout=layout)
py.iplot(fig)




This works, but it is way too inefficient. The documentation does not explain how plotly could generate traces automatically from data columns. I figured this process could be done with a series of for loops, though this is rather complicated. There must be three loops altogether. The first one generates all the traces and put them in a dictionary with the keys being the name of the freedom and the objects being the codes generating the traces. The second append the objects of the dictionary in a data list that corresponds to the data generating function of plotly choropleth. Finally, the third one generates all the buttons for the layout.

First, generate the traces

In [11]:
import plotly.plotly as py
from plotly.graph_objs import *

trace = {}
columnData = data2016.columns[2:13]

for column in columnData:
    trace[column] = Choropleth(
        locations = data2016['code'],
        z = data2016[column],
        text = data2016['name'],
        autocolorscale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Index Scale'),
      )

dataList = []
for key, obj in trace.iteritems():
    dataList.append(obj)

data = Data(dataList)
lengthOfData = len(dataList)

The first for loop returns a choropleth for each column and replaces _z_ with a new data column each loop. I included the variable _lengthOfData_ to be used later to generate a series of boolean values corresponding to the visibility of each data set when an option is selected in the drop down menu.

Now for the layout. I create an empty list for all the buttons. For each button, a list of False values is created with the same amount of element as the length of the dataList. This would hide all traces by default regardless of which option is selected in the dropdown menu. The value with the same index as the selected option however is then changed to True to display the trace. The list of all buttons is then appended to the buttons option in the layout function.

In [12]:
allButtons = []
for index, column in enumerate(columnData):
    trueFalseVis = [False] * lengthOfData
    trueFalseVis[index] = True
    allButtons.append(dict(
                           args=['visible', trueFalseVis],
                           label=column,
                           method='restyle'
                            ))

layout = Layout(
    updatemenus=list([
        dict(
            x=-0.05,
            y=1,
            buttons= list(allButtons),
            yanchor='top'
        ),
        dict(
            x=-0.05,
            y=0.8,
            buttons=list([
                        dict(
                        args=['projection.type', 'orthographic'],
                        label='orthographic',
                        method='restyle'
                        ),
                        dict(
                        args=['projection.type', 'mercator'],
                        label='mercator',
                        method='restyle'
                        )
                    ]))
    ]),
    title = '2016 Index of Economic Freedom<br>Source:\
            <a href="http://www.heritage.org/index/">\
            The Heritage Foundation</a>',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'orthographic'
        )
        ),
)
fig = Figure(data=data, layout=layout)
py.iplot(fig)



There are a few things I have not managed to figure out. The first is that the numbers in the legend are multipled and jumbled up upon being generated, though this problem disappears when another option is selected from the dropdown menu. The second also concerns the legend. Even though all of our scores range from 0 to 100, the legend is generated differently each time for some unknown reason. The third problem concern the hover text. While in the trial graph, the text in the while box displays the ISO-3 code of the selected country, the text in the final graph displays the trace number instead. This is confusing and ideally should not be there at all, but I cannot find any option to turn it off. Lastly, I cannot figure out how to add another button to restyle the visualization from globe to map. It seems that arguments from buttons are automatically passed to data and I am not sure if there is a way to pass them to layout instead. I cannot confirm the possibility of this due to the lack of documentation and available examples for python plotly.

Still, I am quite satisfied with the visualization. It is generic enough now that it can be used with any data sets of similar structure by simply swapping the name of the data set and the title in the function, which I think is a significant improvement from what plotly has in their documentation. 